In [56]:
import pandas as pd
import sqlite3

In [57]:
! ls

README.md
data.sqlite
datasqlite
env
images
index.ipynb
joins_lab.ipynb
splitter.py
venn.png


In [58]:
# Creating connection with the database..
conn = sqlite3.connect('data.sqlite')

# Creating a cursor object to interact with the database
cursor = conn.cursor()

cursor.execute('''SELECT name FROM sqlite_master WHERE type = 'table';''')
table_name = cursor.fetchall()
table_name

[('productlines',),
 ('offices',),
 ('customers',),
 ('employees',),
 ('orderdetails',),
 ('orders',),
 ('payments',),
 ('products',)]

In [59]:
# Here we check the schema of the employees table from the sql database
cursor.execute(""" SELECT sql FROM sqlite_master WHERE type = 'table' AND name = 'employees';""")
employees_C = cursor.fetchall()
employees_C

[('CREATE TABLE "employees" (\n\t"employeeNumber"\tINTEGER,\n\t"lastName"\tTEXT,\n\t"firstName"\tTEXT,\n\t"extension"\tTEXT,\n\t"email"\tTEXT,\n\t"officeCode"\tINTEGER,\n\t"reportsTo"\tINTEGER,\n\t"jobTitle"\tTEXT\n)',)]

In [60]:
employees = pd.read_sql("""
SELECT * FROM employees; """, conn)
employees.head()

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)


In [63]:
offices = """
SELECT * FROM offices;"""
offices_df = pd.read_sql(offices, conn)
offices_df

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan
5,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
6,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA
7,27,Boston,+1 977 299 8345,105 Cambridge Street,,MA,USA,02331,NA


In [67]:
# Select the names of all employees in Boston..
# Join the employees and office table selecting the first and last name
boston = """
SELECT firstName, lastName, jobTitle
FROM employees 
     JOIN offices
     USING(officeCode) 
     WHERE city = 'Paris' AND country = 'France';
"""
pd.read_sql(boston, conn)

,firstName,lastName,jobTitle
0,Gerard,Bondur,Sale Manager (EMEA)
1,Loui,Bondur,Sales Rep
2,Pamela,Castillo,Sales Rep
3,Martin,Gerard,Sales Rep
4,Gerard,Hernandez,Sales Rep


In [41]:
london = """
SELECT firstName, lastName, officeCode
       FROM employees
       JOIN offices
       USING(officeCode)
       WHERE city = 'London';
"""
pd.read_sql(london, conn)


,firstName,lastName,officeCode
0,Larry,Bott,7
1,Barry,Jones,7


In [43]:
# Selecting the names of the employees in USA 
# by joining the employees and office table selecting the first name, last name and office code
query_USA = """
SELECT firstName, lastName, officeCode
       FROM employees
       JOIN offices
       USING(officeCode)
       WHERE country = 'USA';
"""
pd.read_sql(query_USA, conn)

,firstName,lastName,officeCode
0,Anthony,Bow,1
1,Jeff,Firrelli,1
2,Leslie,Jennings,1
3,Diane,Murphy,1
4,Mary,Patterson,1
5,Leslie,Thompson,1
6,Julie,Firrelli,2
7,Steve,Patterson,2
8,Foon Yue,Tseng,3
9,George,Vanauf,3


In [38]:
# Are there any offices that have zero employees?
# Hint: Combine the employees and offices tables and use a group by. 
# Select the office code, city, and number of employees.
Q1 = """
SELECT officeCode, city, COUNT(employeeNumber) AS num_employees
       FROM offices 
       LEFT JOIN employees 
       USING(officeCode)
       GROUP BY officeCode 
       HAVING num_employees = 0;
"""
pd.read_sql(Q1, conn)

,officeCode,city,num_employees
0,27,Boston,0


In [73]:
J = """
SELECT officeCode, City, COUNT(employeeNumber) AS num_employees 
      FROM offices
      LEFT JOIN employees
      USING(officeCode)
      GROUP BY officeCode
      HAVING num_employees = 0;
"""
pd.read_sql(J, conn)

,officeCode,city,num_employees
0,27,Boston,0


In [77]:
# Are there  offices that have 6 employees?
W = """
SELECT officeCode, City, COUNT(employeeNumber) AS num_emplo
      FROM offices 
      LEFT JOIN employees
      USING (officeCode)
      GROUP BY officeCode
      HAVING num_emplo = 6;
"""
dfW = pd.read_sql(W, conn)
dfW

,officeCode,city,num_emplo
0,1,San Francisco,6


In [ ]:
  # Note that COUNT(*) is not appropriate here because
# we are trying to count the _employees_ in each group.
# So instead we count by some attribute of an employee
# record. The primary key (employeeNumber) is a 
# conventional way to do this

In [50]:
customers = """
SELECT * FROM customers;"""
customers_df = pd.read_sql(customers, conn)
customers_df.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,,Stavern,,4110,Norway,1504,81700


In [53]:
offices_df.head()

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan


## Questions to ask while dealing with the database..

In [54]:
# How many customers are there per office?
cust = """
SELECT officeCode, city, COUNT(customerNumber) AS num_customers
       FROM offices
       LEFT JOIN customers
       USING(postalCode)
       GROUP BY postalCode;
"""
#pd.read_sql(cust, conn)

In [97]:
cust = """
SELECT oc officeCode , c city , COUNT(cn customerNumber) AS num_customers
       FROM offices AS o
       JOIN employees AS e
            USING(officeCode)
       JOIN customers AS c
            ON e.employeeNumber = c.salesRepEmployeeNumber
       GROUP BY officeCode;
"""
#df_cust = pd.read_sql(cust, conn)         

In [95]:
q = """
SELECT
    o.officeCode,
    o.city,
    COUNT(c.customerNumber) AS n_customers
FROM offices AS o
JOIN employees AS e
    USING(officeCode)
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY officeCode
;
"""
pd.read_sql(q, conn)

,officeCode,city,n_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17
